# Step One: Scraping

Initial scraping using Notebook, BeautifulSoup, Pandas & Requests/Splinter

In [1]:
#import things
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests

## Nasa Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [2]:
#define a variable to store the variable for the website

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [3]:
# identify location of chromedriver and store it as a variable
driverPath = !which chromedriver

# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': driverPath[0]}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#inform the browser to visit the page
browser.visit(url)

In [5]:
# Splinter can capture a page's underlying html and use pass it to BeautifulSoup to allow us to scrape the content
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Using BS, we can execute standard functions to capture the page's content
news_title = soup.find_all('div', class_='content_title')[1].string
news_p = soup.find_all('div', class_='article_teaser_body')[0].text 

print(news_title)
print('------')
print(news_p)

NASA's Perseverance Rover Is Midway to Mars 
------
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


## JPL Mars Space Images - Featured Images

In [6]:
#define the variable to store the url for landing image

image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(image_url)

In [7]:
# Splinter can capture a page's underlying html and use pass it to BeautifulSoup to allow us to scrape the content
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# print(soup.prettify())


In [8]:
#try to get all of the html related to images
images = soup.find_all('a', class_='fancybox')
images

large_image = []
for image in images:
    photo = image['data-fancybox-href']
    large_image.append(photo)
    
large_image

featured_image_url = 'https://www.jpl.nasa.gov' + photo

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24118_hires.jpg


## Mars Facts

Visit Mars Facts webpage and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

Use Pandas to convert the data to a HTML table string

In [9]:
#define a variable to to the url for mars facts

url = 'https://space-facts.com/mars/'

In [41]:
#read html page with pandas as requested

facts = pd.read_html(url)
len(facts)
#print out all of the fact tables to see which one I need
facts

# #table 3 is the one needed 

facts[2]

#set into a dataframe
df = facts[2]
df.keys()

df.set_index(0, inplace=True)
df
df.index.name = None
#rename the columns
df.columns = ['Mars']
df


,Mars
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [50]:
#generate html file from dataframe
html_table = df.to_html()
print(html_table)

#save as html file
table_html = df.to_html('table.html')

!open table.html

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

Visit the USGS Astrogeology site to obtain hi-res images for each of Mar's hemispheres

<ul> You will need to click each link to the hemispheres to find the image url for the full-res image </ul>

<ul> Save both image url string for the full res hemisphere image, and the Hemisphere title containing the hemisphere name. Use Python dictioniary to store the data using the keys img_url and title</ul>

<ul> Append the dictionary with the image url string and hemisphere time to a list. This list will contain one dictionary for each hemisphere. </ul>

In [43]:
#get the url 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

#use BS to bring in the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#create the list
mars_urls = []

#retrieve all elements that contain the information needed 
results = soup.find('div', class_='result-list')
each_hemisphere = results.find_all('div', class_='item')

In [44]:
#create a for loop to get all the information
for i in each_hemisphere:
    hemi_title = i.find('h3').text
    each_link = i.find('a')['href']
    img_url = 'https://astrogeology.usgs.gov/' + each_link
#     get the link to visit each page and get the image 
    browser.visit(img_url)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    each_pg = soup.find('div', class_='downloads')
    image = each_pg.find('a')['href']
    mars_urls.append({'title': hemi_title, 'img_url': image})

In [47]:
print(mars_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [49]:
#double check to see the urls and put them into a dictionary:

mars_info = {
    'nasa_news': news_title, 
    'nasa_p': news_p,
    'featured_img_url': featured_image_url,
    'mars_facts': html_table, 
    'hemisphere_urls': mars_urls
}

print(mars_info)

{'nasa_news': "NASA's Perseverance Rover Is Midway to Mars ", 'nasa_p': "Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.", 'featured_img_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24118_hires.jpg', 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 day